In [10]:
from neo4j import GraphDatabase
from langchain.chains import GraphQAChain
from langchain.chat_models import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.vectorstores import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate

In [3]:
NEO4J_URL = ""
NEO4J_USERNAME = ""
NEO4J_PASSWORD = ""
NEO4J_DATABASE = ""
openai_key = ""

driver = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

kg = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=NEO4J_DATABASE
)

In [4]:
cypher = """
    MATCH (n)
    WHERE ANY(label IN labels(n) WHERE label IN ['Condition', 'Insurance', 'Medication', 'Attribute'])
    SET n:Task
    RETURN COUNT(n) AS nodes_labeled
    """

In [5]:
kg.query(cypher)

[{'nodes_labeled': 27}]

In [13]:
vector_store = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(openai_api_key=openai_key),
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=NEO4J_DATABASE,
    index_name="vector_index"
    node_label="Task",
    text_node_properties=["name", "description"],
    embedding_node_property="embedding"
)

# Create a retriever and Q&A chain
retriever = vector_store.as_retriever()

custom_prompt = PromptTemplate(
    input_variables=["page_content"],
    template="{page_content}\n\n"
)

qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=ChatOpenAI(temperature=0, openai_api_key=openai_key, model="gpt-4"),
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"document_prompt": custom_prompt}
)

In [14]:
question = "What brand of PCSK9 inhibitors will cause loss of appetite? If you are unsure about the answer, return 'I don't know'. Do not say anything else."
qa_chain(
    {"question": question},
    return_only_outputs=True,
)

{'answer': 'The brand of PCSK9 inhibitors that may cause loss of appetite is Repatha.\n',
 'sources': ''}

In [18]:
question = "What brand of PCSK9 inhibitors will cause stomach pain? If you are unsure about the answer, return 'I don't know'. Do not say anything else."
qa_chain(
    {"question": question},
    return_only_outputs=True,
)

{'answer': 'The brand of PCSK9 inhibitors that may cause stomach pain is Repatha.\n',
 'sources': ''}